In [11]:
# Code Modularity
# Install required Libraries
!pip install --upgrade openai
import os
import re
import time
import json
from getpass import getpass
from typing import List, Dict, Any

In [7]:
# Secure key input
if not os.environ.get("GROQ_API_KEY"):
    groq_key = getpass("Provide GROQ API key: ").strip()
    if groq_key:
        os.environ["GROQ_API_KEY"] = groq_key
        print("GROQ_API_KEY set in runtime")
    else:
        print("No key provided — code will run with local fallbacks")

# Import the OpenAI-compatible client class
try:
    from openai import OpenAI
    client = None
    if os.environ.get("GROQ_API_KEY"):
        client = OpenAI(api_key=os.environ.get("GROQ_API_KEY"), base_url="https://api.groq.com/openai/v1")
        print("Client configured for Groq.")
    else:
        print("Running without remote client.")
except Exception as e:
    print("OpenAI client import/config failed:", e)
    client = None


Client configured for Groq.


In [12]:
# Task 1 - ConversationManager
# Manages chat history, truncation, and periodic summarization

class ConversationManager:
    def __init__(self,
                 summarizer = None,               # Function to summarize conversation
                 summarization_interval: int = 3, # Summarize after every k messages
                 max_turns: int = None,           # Limit by number of messages
                 max_chars: int = None,           # Limit by total characters
                 max_words: int = None):          # Limit by total words
        """
        Initializes conversation manager with options for truncation and summarization.
        """
        self.history: List[Dict[str,str]] = []   # Stores message dicts:
        self.summarizer = summarizer
        self.summarization_interval = summarization_interval or 0
        self.max_turns = max_turns
        self.max_chars = max_chars
        self.max_words = max_words
        self.turn_count = 0                     # Tracks total messages added
        self.summarized_blob = None             # Stores last summary if created

    def add_message(self, role: str, content: str):
        """
        Add a single message to the conversation.
        Role can be 'user', 'assistant', 'system', or 'summary'.
        Handles truncation and periodic summarization.
        """
        assert role in ("user", "assistant", "system", "summary"), "Invalid role."
        self.history.append({"role": role, "content": content})
        self.turn_count += 1

        # Apply truncation rules if set
        self._apply_truncation()

        # Periodic summarization if k-th message
        if self.summarization_interval and (self.turn_count % self.summarization_interval == 0):
            self._perform_summarization()

    def _apply_truncation(self):
        """
        Internal method to truncate history by turns, chars, or words.
        """
        # Truncate by turns (keep only last N messages)
        if self.max_turns is not None and len(self.history) > self.max_turns:
            self.history = self.history[-self.max_turns:]

        # Truncate by characters (keep total under max_chars)
        if self.max_chars is not None:
            total = sum(len(m["content"]) for m in self.history)
            while total > self.max_chars and len(self.history) > 1:
                removed = self.history.pop(0)
                total -= len(removed["content"])

        # Truncate by words (keep total under max_words)
        if self.max_words is not None:
            total_words = sum(len(m["content"].split()) for m in self.history)
            while total_words > self.max_words and len(self.history) > 1:
                removed = self.history.pop(0)
                total_words -= len(removed["content"].split())

    def _perform_summarization(self):
        """
        Internal method to summarize the current conversation using the provided summarizer.
        Replaces history with a single summary message.
        """
        if not self.summarizer:
            return  # Skip if no summarizer provided

        # Prepare full conversation text for summarization
        full_text = "\n".join(f'{m["role"]}: {m["content"]}' for m in self.history)
        try:
            summary = self.summarizer(full_text)
        except Exception as e:
            print("Summarizer error:", e)
            return

        # Store summary and replace history with it
        self.summarized_blob = summary
        self.history = [{"role": "summary", "content": summary}]

    def get_history(self, last_n: int = None, max_chars: int = None, max_words: int = None) -> List[Dict[str,str]]:
        """
        Return a copy of the current conversation history.
        Optional truncation parameters:
            last_n: return only last N messages
            max_chars: keep total under this many characters
            max_words: keep total under this many words
        """
        hist = list(self.history)

        # Truncate by last_n messages
        if last_n is not None:
            hist = hist[-last_n:]

        # Truncate by characters
        if max_chars is not None:
            out = []
            total = 0
            for m in reversed(hist):
                c = len(m["content"])
                if total + c > max_chars and out:
                    break
                out.append(m)
                total += c
            hist = list(reversed(out))

        # Truncate by words
        if max_words is not None:
            out = []
            total = 0
            for m in reversed(hist):
                w = len(m["content"].split())
                if total + w > max_words and out:
                    break
                out.append(m)
                total += w
            hist = list(reversed(out))

        return hist

    def show(self, pretty = True):
        """
        Print conversation history in a readable format.
        """
        if pretty:
            for m in self.history:
                print(f"{m['role'].upper()}: {m['content']}")
        else:
            return self.history


In [13]:
# Defining Summarizer functions
# Provides two options:
# 1. model_summarizer - uses Groq/OpenAI-compatible client
# 2. local_fallback_summarizer - simple extractive summarization

def local_fallback_summarizer(text: str, max_sentences: int = 3) -> str:
    """
    Local fallback summarizer (extractive).
    Splits text into sentences and returns the first few.
    """
    # Split text into sentences using punctuation
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # Keep only non-empty sentences
    sentences = [s.strip() for s in sentences if s.strip()]
    # Return first max_sentences sentences
    return " ".join(sentences[: max_sentences]) if sentences else text[: 300]

def model_summarizer(text: str, model_name: str = "llama-3.3-70b-versatile", max_tokens: int = 300) -> str:
    """
    Uses Groq/OpenAI-compatible client to summarize text.
    If client is not available, falls back to local_summarizer.
    """
    # Use local fallback if client is not configured
    if client is None:
        return local_fallback_summarizer(text)

    # Prepare messages for chat completion
    messages = [
        {"role":"system", "content":"You will be a concise summarizer. Produce a short paragraph for summarizing the conversation"},
        {"role":"user", "content": f"Summarize this conversation history:\n\n{text}"}
    ]

    try:
        # Call Groq model via OpenAI-compatible client
        response = client.chat.completions.create(
            model = model_name,
            messages = messages,
            max_tokens = max_tokens,
            temperature = 0.2
        )

        # Extract summary from response
        choice = response.choices[0]
        summary_text = ""

        # Try standard attribute
        if hasattr(choice, "message") and getattr(choice.message, "content", None):
            summary_text = choice.message.content
        else:
            # Dict-like fallback
            try:
                summary_text = choice["message"]["content"]
            except Exception:
                # Try 'text' field as a last resort
                summary_text = getattr(choice, "text", None) or (choice.get("text") if isinstance(choice, dict) else "")

        return (summary_text or "").strip()

    except Exception as e:
        # In case of failure, use local fallback
        print("Model summarizer failed, using local fallback. Error:", e)
        return local_fallback_summarizer(text)


In [14]:
# Demonstration:
# 1. Adding multiple conversation messages
# 2. Truncation options
# 3. Periodic summarization every k-th message

# Instantiate the ConversationManager
# - summarizer: model_summarizer (uses Groq client if available)
# - summarization_interval: 3 (summarize after every 3 messages)
cm = ConversationManager(
    summarizer = model_summarizer,
    summarization_interval = 3,
    max_turns = None,   # No turn limit for this demo
    max_chars = None,   # No character limit for this demo
    max_words = None    # No word limit for this demo
)

# Sample conversation messages (role, content)
samples = [
    ("user", "Hi, I'm Omkar Patil. I want to join your newsletter and get updates."),
    ("assistant", "Welcome Omkar! Can you Provide your email and phone?"),
    ("user", "Email omkarpatil1100@gmail.com, phone +91-9035221719. I'm based in Banglore and age 26."),
    ("assistant", "Thanks Omkar — I saved your contact. Do you prefer weekly or monthly updates?"),
    ("user", "Weekly please. Also interested in data science articles."),
    ("assistant", "Great: weekly on data science. Anything else we should note?"),
    ("user", "No, that's it. Thank you!")
]

# Add messages one by one and print history after each
print("Adding messages one by one and showing conversation history after each message...\n")
for i, (role, msg) in enumerate(samples, 1):
    # Add message to conversation
    cm.add_message(role, msg)

    # Print current state of conversation
    print(f"\n--- After message #{i} ({role}) ---")
    cm.show(pretty = True)

    # If a summary was created (every k-th message), show it
    if cm.summarized_blob:
        print("\n[Summarized blob stored in history:]\n", cm.summarized_blob[:400])

    # Small delay for readability in demo
    time.sleep(0.2)


Adding messages one by one and showing conversation history after each message...


--- After message #1 (user) ---
USER: Hi, I'm Omkar Patil. I want to join your newsletter and get updates.

--- After message #2 (assistant) ---
USER: Hi, I'm Omkar Patil. I want to join your newsletter and get updates.
ASSISTANT: Welcome Omkar! Can you Provide your email and phone?

--- After message #3 (user) ---
SUMMARY: Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newsletter and receive updates, providing his email (omkarpatil1100@gmail.com) and phone number (+91-9035221719) for subscription.

[Summarized blob stored in history:]
 Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newsletter and receive updates, providing his email (omkarpatil1100@gmail.com) and phone number (+91-9035221719) for subscription.

--- After message #4 (assistant) ---
SUMMARY: Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newslett

In [24]:
# Truncation Options
# To retrieve a subset of the conversation history based on different truncation settings:
#   1. last_n messages
#   2. max_chars
#   3. max_words

# Show the full current conversation history first
print("\n Full conversation history")
cm.show()  # Prints all messages in history

# Truncate by last_n messages, Keeps only the last 2 messages from history
print("\n Last 2 messages only")
last_two = cm.get_history(last_n = 2)
for msg in last_two:
    print(msg["role"].upper() + ":", msg["content"])

# Truncate by max_chars, Keeps messages such that total characters do not exceed 180
print("\n Last 180 characters only")
last_180_chars = cm.get_history(max_chars = 180)
for msg in last_180_chars:
    print(msg["role"].upper() + ":", msg["content"])

# Truncate by max_words, Keeps messages such that total words do not exceed 80
print("\n Last 80 words only")
last_80_words = cm.get_history(max_words = 80)
for msg in last_80_words:
    print(msg["role"].upper() + ":", msg["content"])



 Full conversation history
SUMMARY: Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newsletter, providing his email and phone number. He expressed interest in receiving weekly updates, specifically on data science articles, and his preferences have been noted.
USER: No, that's it. Thank you!

 Last 2 messages only
SUMMARY: Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newsletter, providing his email and phone number. He expressed interest in receiving weekly updates, specifically on data science articles, and his preferences have been noted.
USER: No, that's it. Thank you!

 Last 180 characters only
USER: No, that's it. Thank you!

 Last 80 words only
SUMMARY: Omkar Patil, a 26-year-old from Bangalore, initiated a conversation to join a newsletter, providing his email and phone number. He expressed interest in receiving weekly updates, specifically on data science articles, and his preferences have been noted.
USER: No, tha